In [36]:

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.feeds as btfeed



class setUpTrigger():
    
    def __init__(self, stoch):
        self.stoch = stoch
        self.stochk = stoch.slowk[0]
        self.stochd = stoch.slowd[0]
        
        self.status = self.stochastic_logic()
        self.msg = "Message goes here"
        
    def stochastic_logic(self):
        # BUY and SELL Stochastic Logic
        if self.stoch.slowk[-1] < self.stoch.slowd[-1] and self.stoch.slowk[0] > self.stoch.slowd[0]:
            return "buy"

        elif self.stoch.slowk[-1] > self.stoch.slowd[-1] and self.stoch.slowk[0] < self.stoch.slowd[0]:
            return "sell"
        
        
        
class moneyManagement():
    
    def __init__(self, position , data):
        self.position = position
        self.dataclose = data.close
        self.datahigh = data.high
        self.datalow = data.low
        
    def exitcommand(self):
        
        # Exit command
        if self.position == "buy" and self.datalow[0] > self.dataclose[0]:
            print("Custom Close called")
            return True
        elif self.position == "sell"  and self.datahigh[0] < self.dataclose[0]:
            print("Custom Close called")
            return True
        else:
            return False
        
    
    



class forexSpreadCommisionScheme(bt.CommInfoBase):
    '''
    This commission scheme attempts to calcuate the commission hidden in the
    spread by most forex brokers. It assumes a mid point data is being used.

    *New Params*
    spread: Float, the spread in pips of the instrument
    JPY_pair: Bool, states whether the pair being traded is a JPY pair
    acc_counter_currency: Bool, states whether the account currency is the same
    as the counter currency. If false, it is assumed to be the base currency
    '''
    params = (
        ('spread', 2),
        ('stocklike', False),
        ('JPY_pair', False),
        ('acc_counter_currency', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    
    def _getcommission(self, size, price, pseudoexec):
        '''
        This scheme will apply half the commission when buying and half when selling.
        If JPY pair change the multiplier accordingly.
        If account currency is same as the base currency, change pip value calc.
        '''
        if self.p.JPY_pair == True:
            multiplier = 0.01
        else:
            multiplier = 0.0001

        if self.p.acc_counter_currency == True:
            comm = abs((self.p.spread * (size * multiplier)/2))

        else:
            comm =  abs((self.p.spread * ((size / price) * multiplier)/2))

        return comm
    
    
    
    
    
    
    
    
    
    
    
    
    
    
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

        
        
        
        
    def __init__(self):
        self.count = 0

        
        
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

        
        
        
        
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        print(('OPERATION PROFIT, GROSS {}, NET {}'.format(trade.pnl, trade.pnlcomm)))

        
        
        
        
        
    
    
    def next(self):
        self.count = len(self)
        if self.count == 1:
            self.buy()
        elif self.count == 10:
            self.close()
        else:
            pass
        
            
            

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    
    #Variable for our starting cash
    startcash = 100
    
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=50)
    
    # Set our desired cash start
    cerebro.broker.setcash(startcash)


    # Set the commission
    #Add the new commissions scheme
    comminfo = forexSpreadCommisionScheme(spread=10, acc_counter_currency=False)
    cerebro.broker.addcommissioninfo(comminfo)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Plot the result
    cerebro.plot()

 

Starting Portfolio Value: 100.00
2019-01-01, BUY EXECUTED, Price: 1.15, Cost: 50.00, Comm 0.02
2019-01-02, SELL EXECUTED, Price: 1.15, Cost: 50.00, Comm 0.02
OPERATION PROFIT, GROSS 0.06950000000000012, NET 0.025898311424388434
